# Sample usage of classified images and snow cover statistics

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import xarray as xr
import rioxarray as rxr

## Define paths in directory

In [ ]:
# If using Google Colab, mount Google Drive to access your files
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Define base path to data for convenience
data_path = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/study-sites/'

# Define RGI ID for glacier
rgi_id = 'RGI60-01.00046'

# Grab all classified image file names
im_classified_fns = sorted(glob.glob(os.path.join(data_path, rgi_id, 'imagery', 'classified', '*_classified.nc')))

# Grab all snow cover statistics file names
scs_fn = os.path.join(data_path, rgi_id, rgi_id + '_snow_cover_stats.csv')

## Analyze/plot classified images

In [ ]:
# Load one classified image and display
im_classified_fn = im_classified_fns[0]
im_classified = xr.open_dataset(im_classified_fn)
im_classified

In [ ]:
# Define a color map for plotting classified images
colors = ["#4eb3d3", # 0 = Snow
          "#636363", # 1 = Shadowed snow
          "#084081", # 2 = Ice
          "#fe9929", # 3 = Rock/debris
          "#252525"  # 4 = Water
         ]
cmap_classified = matplotlib.colors.ListedColormap(colors)
cmap_binary = matplotlib.colors.ListedColormap(['w', '#6a51a3'])

plt.rcParams.update({'font.size':12, 'font.sans-serif':'Arial'})

# Iterate over a few classified images and plot
for im_classified_fn in im_classified_fns[75:80]:
    # Load classified image
    im_classified = xr.open_dataset(im_classified_fn)

    # Grab image date and image source for title
    date = str(im_classified.time.data[0])[0:10]
    source = os.path.basename(im_classified_fn).split(rgi_id + '_')[1].split('_classified')[0]

    # Remove no data values
    im_classified = xr.where(im_classified==-9999, np.nan, im_classified)
    
    # Plot
    fig, ax = plt.subplots(1, 2, figsize=(12,4), sharey=True)
    # classified image
    im = ax[0].imshow(im_classified['classified'].data[0], cmap=cmap_classified, clim=(1, 5.5),
                      extent=(np.min(im_classified.x.data), np.max(im_classified.x.data),
                              np.min(im_classified.y.data), np.max(im_classified.y.data)))
    ax[0].set_title('Classified image')
    cbar = fig.colorbar(im, ax=ax[0], orientation='horizontal')
    cbar.ax.set_xticks([1.4, 2.3, 3.2, 4.1, 5.0])
    cbar.ax.set_xticklabels(['Snow', 'Shadowed snow', 'Ice', 'Rock/Debris', 'Water']) 
    # binary snow image
    im_classified_binary = xr.where(im_classified <= 2, 1, 0)
    im = ax[1].imshow(im_classified_binary['classified'].data[0], cmap=cmap_binary, clim=(0, 1),
                      extent=(np.min(im_classified.x.data), np.max(im_classified.x.data),
                              np.min(im_classified.y.data), np.max(im_classified.y.data)))
    cbar = fig.colorbar(im, ax=ax[1], orientation='horizontal', shrink=0.6)
    cbar.ax.set_xticks([0.25, 0.75])
    cbar.ax.set_xticklabels(['Snow-free', 'Snow-covered']) 
    ax[1].set_title('Binary snow image')
    
    fig.suptitle(f'{date} {source}')
    fig.tight_layout()
    plt.show()

In [ ]:
## Add a cell for pulling snow/no-snow along a line over time

## Analyze/plot snow cover statistics

In [ ]:
# Load snow cover statistics and view
scs = pd.read_csv(scs_fn)
scs['datetime'] = pd.to_datetime(scs['datetime'])
scs

In [ ]:
# Plot time series for a few columns
columns = ['snowline_elevs_median_m', 'SCA_m2', 'AAR', 'ELA_from_AAR_m']
columns_display = ['Median snowline elevation [m]', 'Snow-covered area [m$^2$]', 'Accumulation area ratio (AAR)', 'Snowline elevation from AAR [m]']

fig, ax = plt.subplots(len(columns), 1, figsize=(10, 4*len(columns)), sharex=True)

for i in range(0,len(columns)):
    ax[i].plot(scs['datetime'], scs[columns[i]], '.', markersize=5)
    ax[i].set_title(columns_display[i])
    ax[i].grid()

plt.show()